In [26]:
# ---------------
# Imports
# ---------------

!pip install googlesearch-python
import pandas as pd
from googlesearch import search
import requests
from bs4 import BeautifulSoup
import spacy
from spacy import displacy
# import time

# pd.options.display.max_colwidth = 1000 # for viewing text
NER = spacy.load("en_core_web_sm")

# Mount drive - needed for GBQ creds
# from google.colab import drive
# drive.mount('/content/drive')

# ---------
# Data
# ---------

imdb_reviews = pd.read_csv('https://raw.githubusercontent.com/naserahmadi/TDmatch/main/data/imdb/IMDB_reviews.csv', names = ['text'])
imdb_reviews['row_number'] = imdb_reviews.reset_index().index

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [48]:
# ------------------------------
# NER & functions to Google
# ------------------------------

def run_NER(text, text_id):

  ner_text = NER(text)
  ner_result = [pd.DataFrame({'id': text_id, 'word': x.text, 'entity_detected': x.label_}, index = [0]) for x in ner_text.ents]

  return ner_result


def url_title(url):

  try:
    reqs = requests.get(url)
    soup = BeautifulSoup(reqs.text, 'html.parser')
    url_title = soup.find('title').get_text()

  except:
    url_title = ''

  return url_title


def title_NER(title):

  try:
    ner_results = NER(title)
    ner_df = [pd.DataFrame({'word': x.text, 'entity_detected': x.label_}, index = [0]) for x in ner_results.ents]
    ner_all = pd.concat(ner_df)

    return ner_all

  except:
    pass  

In [34]:
# displacy.render(text2, style = "ent", jupyter = True)

# NER on unstructured text, Googling entities

In [49]:
tryme = imdb_reviews.iloc[4] # select 4th row to test
tryme_res = run_NER(tryme.text, tryme.row_number)
tryme_df = pd.concat(tryme_res)
tryme_unique = tryme_df.drop_duplicates()
tryme_search = tryme_unique.groupby(['id'])['word'].apply(' '.join).reset_index()
tryme_google = search(tryme_search.word.values[0])

# Feed entities into Google
google_results = list(tryme_google)
article_titles = list(map(url_title, google_results))

# NER on Google titles
title_entities = list(map(title_NER, article_titles))
title_ents_df = pd.concat(title_entities)
title_ents_df.query('entity_detected in ["ORG", "PERSON", "WORK_OF_ART"]').groupby('word').count()

# Optional: parse URL and return full text for NER/majority voting

In [89]:
# !pip install newspaper3k
# import newspaper
# import json

In [90]:
# article = newspaper.Article(url = 'https://en.wikipedia.org/wiki/The_Dark_Knight_Rises', language='en')
# article.download()
# article.parse()

In [91]:
# article ={
#     "title": str(article.title),
#     "text": str(article.text),
#     "authors": article.authors,
#     "published_date": str(article.publish_date),
#     "top_image": str(article.top_image),
#     "videos": article.movies,
#     "keywords": article.keywords,
#     "summary": str(article.summary)
# }

In [95]:
dk_knight = NER(article['text'])

In [101]:
dk_knight_entities = [pd.DataFrame({'word': x.text, 'entity_detected': x.label_}, index = [0]) for x in dk_knight.ents]